In [3]:
!pip install diffusers pillow transformers accelerate

  Using cached diffusers-0.35.1-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
   ------- -------------------------------- 0.8/4.1 MB 3.7 MB/s eta 0:00:01
   ------- -------------------------------- 0.8/4.1 MB 3.7 MB/s eta 0:00:01
   ------------------------- -------------- 2.6/4.1 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 4.1/4.1 MB 5.5 MB/s eta 0:00:00


In [ ]:
import torch
from PIL import Image
from transformers import pipeline
from diffusers import StableDiffusionPipeline

device = "cuda" if torch.cuda.is_available() else "cpu"

# --- A) IMAGE -> CAPTION ------------------------------------------------------
# Simple, small captioner (fast to play with). Alternatives below.
cap = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning", device=0 if device=="cuda" else -1)

img_path = "input.jpg"  # your image
img = Image.open(img_path).convert("RGB")

# Beam search helps a bit over pure greedy
caption_outputs = cap(img, max_new_tokens=32)
caption = caption_outputs[0]["generated_text"]
print("Caption:", caption)

# --- B) CAPTION -> IMAGE ------------------------------------------------------
# Stable Diffusion (v1-5 is light; SDXL also works if you have VRAM)
sd = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
)
sd = sd.to(device)
sd.enable_attention_slicing()  # helps memory on smaller GPUs

# Settings you can tweak
seed = 1234
g = torch.Generator(device=device).manual_seed(seed)
num_inference_steps = 30
guidance_scale = 7.5

out = sd(
    prompt=caption,
    num_inference_steps=num_inference_steps,
    guidance_scale=guidance_scale,
    generator=g,
)
gen = out.images[0]
gen.save("reconstructed_from_caption.png")
print("Saved: reconstructed_from_caption.png")


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Device set to use cpu
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Caption: a table with chairs and a table cloth 


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

c:\Users\vlad\miniconda3\envs\nlp\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vlad\.cache\huggingface\hub\models--runwayml--stable-diffusion-v1-5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]